In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


mc = MongoClient()
capstone_db = mc['bit_scraping']
#artist_names  = capstone_db['artist_names']
artist_info = capstone_db['artist']

In [2]:
#gd.download_band_names(['OR'],'bands_in_ca.csv')

In [3]:
#gd.download_band_names(['OR','CA'],'bands_in_ca.csv')

In [4]:
#artist_info.delete_many({})

In [5]:
data = pd.read_csv('events.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103427 entries, 0 to 103426
Data columns (total 30 columns):
Unnamed: 0              103427 non-null int64
_id                     103427 non-null object
artist_facebook         64697 non-null object
artist_image            103427 non-null object
artist_name             103427 non-null object
artist_thumb            103427 non-null object
artist_url              103427 non-null object
artists_id              103427 non-null int64
bio                     77455 non-null object
genre                   85050 non-null object
home_town               64764 non-null object
mbid                    41035 non-null object
tracker_count           103427 non-null int64
upcoming_event_count    103427 non-null int64
offers                  103427 non-null object
datetime                103427 non-null object
on_sale_datetime        34191 non-null object
description             32091 non-null object
lineup                  103427 non-null object
event_i

In [7]:
df = data[['artists_id', 'venue_id']]

In [8]:
df.head(3)

,artists_id,venue_id
0,14849750,12242
1,14849750,15607
2,14849750,23800


In [9]:
df['event_counts'] = 1

/Users/shiahlints/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head(5)

,artists_id,venue_id,event_counts
0,14849750,12242,1
1,14849750,15607,1
2,14849750,23800,1
3,14849750,4271,1
4,14849750,16032,1


In [11]:
ratings_matrix = df.groupby(['artists_id', 'venue_id'], as_index = False)

In [12]:
dt = pd.DataFrame(ratings_matrix.count())

In [16]:
dt.event_counts = 1

In [17]:
dt.head(5)

,artists_id,venue_id,event_counts
0,4,2145,1
1,4,18590,1
2,4,28176,1
3,7,1944,1
4,7,3203,1


In [18]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [19]:
sp_df = spark.createDataFrame(dt)

In [20]:
sp_df.show()

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         4|    2145|           1|
|         4|   18590|           1|
|         4|   28176|           1|
|         7|    1944|           1|
|         7|    3203|           1|
|         7|    5160|           1|
|         7|    7045|           1|
|         7|   12463|           1|
|         7|   13845|           1|
|         7|   14862|           1|
|         7|   15279|           1|
|         7|   17487|           1|
|         7|   17913|           1|
|         7|   19230|           1|
|         7|   20502|           1|
|         7|   22184|           1|
|         7|   22796|           1|
|         7|   23130|           1|
|         7|   23274|           1|
|         7|   25258|           1|
+----------+--------+------------+
only showing top 20 rows



In [21]:
train, test = sp_df.randomSplit([0.8, 0.2], seed=42)

In [22]:
als_model = ALS(userCol='venue_id',
                itemCol='artists_id',
                ratingCol='event_counts',
                nonnegative=True,
                regParam=0.1,
                rank=5,
                implicitPrefs=True,
                alpha = 40)

In [23]:
recomender = als_model.fit(train)

In [24]:
rjj = 11270709

In [25]:
venues_in_seattle = np.unique(data[data['city'] == 'Seattle']['venue_id'])

In [26]:
rate_this = pd.DataFrame()

In [28]:
rate_this['venue_id'] = venues_in_seattle

In [29]:
rate_this['artists_id'] = rjj

In [30]:
rate_this.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
venue_id      667 non-null int64
artists_id    667 non-null int64
dtypes: int64(2)
memory usage: 10.5 KB


In [31]:
rate_this = rate_this[['artists_id', 'venue_id']]

In [32]:
rate_this_spark = spark.createDataFrame(rate_this)

In [33]:
rate_this_spark.show()

+----------+--------+
|artists_id|venue_id|
+----------+--------+
|  11270709|     138|
|  11270709|     200|
|  11270709|     205|
|  11270709|     265|
|  11270709|     355|
|  11270709|     367|
|  11270709|     392|
|  11270709|     393|
|  11270709|     394|
|  11270709|     397|
|  11270709|     448|
|  11270709|     919|
|  11270709|    1105|
|  11270709|    1130|
|  11270709|    1198|
|  11270709|    1526|
|  11270709|    1647|
|  11270709|    1676|
|  11270709|    1808|
|  11270709|    1858|
+----------+--------+
only showing top 20 rows



In [34]:
predictions = recomender.transform(rate_this_spark)

In [35]:
predictions.show()

+----------+--------+-------------+
|artists_id|venue_id|   prediction|
+----------+--------+-------------+
|  11270709|   18800| 7.7523536E-4|
|  11270709|   29601| 0.0025277736|
|  11270709|   37263|          0.0|
|  11270709|   38422|          0.0|
|  11270709|   40107|          0.0|
|  11270709|   40386|          0.0|
|  11270709|     392|1.0543671E-15|
|  11270709|    8105|          0.0|
|  11270709|   19758|          0.0|
|  11270709|   19868|          0.0|
|  11270709|   32680|          NaN|
|  11270709|    4239|          0.0|
|  11270709|   13465|          0.0|
|  11270709|   23367|          0.0|
|  11270709|   14944|          0.0|
|  11270709|   31646|          0.0|
|  11270709|    2247|          0.0|
|  11270709|   14365|          NaN|
|  11270709|   21126|          0.0|
|  11270709|   17903|          0.0|
+----------+--------+-------------+
only showing top 20 rows



In [36]:
venues = pd.read_csv('venues.csv')

In [37]:
thing = predictions.dropna().sort('prediction',ascending=False)

In [38]:
predictionspd = thing.toPandas()

In [39]:
top_pridictions = predictionspd['venue_id'].values[0:10]

In [40]:
venues[venues['venue_id']== 4239]

,Unnamed: 0,venueidentifer,venue_name,country,region,city,genre,bio,latitude,longitude,venue_id
4239,4239,Black Lodge Seattle WA United States,Black Lodge,United States,WA,Seattle,"['Space Rock, Alternative, Post Hardcore']",['When great bands break up there is rarely an...,47.606209,-122.332071,4239


In [41]:
data[data['venue_id'].isin(top_pridictions)]

,Unnamed: 0,_id,artist_facebook,artist_image,artist_name,artist_thumb,artist_url,artists_id,bio,genre,...,event_url,band_name,venue_name,country,region,city,latitude,longitude,venueidentifer,venue_id
57,57,5b771746482deb63557f8b8c,https://www.facebook.com/victorwootenmusic,https://s3.amazonaws.com/bit-photos/large/6364...,Victor Wooten,https://s3.amazonaws.com/bit-photos/thumb/6364...,https://www.bandsintown.com/a/7792?came_from=2...,7792,“I like to talk and I like to play.”So said Vi...,"Funk, Jazz",...,https://www.bandsintown.com/e/100121031?app_id...,Victor Wooten,Nectar Lounge,United States,WA,Seattle,47.606209,-122.332071,Nectar Lounge Seattle WA United States,23325
200,200,5b7717aa482deb63557f8be1,NaN,https://s3.amazonaws.com/bit-photos/artistLarg...,"The Sea, The Sea",https://s3.amazonaws.com/bit-photos/artistThum...,https://www.bandsintown.com/a/1862270?came_fro...,1862270,NaN,NaN,...,https://www.bandsintown.com/e/1010693119?app_i...,"The Sea, The Sea",Tractor Tavern,United States,WA,Seattle,47.665771,-122.382645,Tractor Tavern Seattle WA United States,38718
223,223,5b7717b5482deb63557f8bf0,https://www.facebook.com/theseatheseamusic,https://s3.amazonaws.com/bit-photos/large/8544...,The Sea The Sea (US),https://s3.amazonaws.com/bit-photos/thumb/8544...,https://www.bandsintown.com/a/11181554?came_fr...,11181554,THE SEA THE SEA is an Upstate New York based i...,"Indie Folk Duo, Indie Folk Americana, Indie Fo...",...,https://www.bandsintown.com/e/1010064178?app_i...,The Sea The Sea (US),Tractor Tavern,United States,WA,Seattle,47.665771,-122.382645,Tractor Tavern Seattle WA United States,38718
1023,1023,5b7718d5482deb63557f8d29,https://www.facebook.com/LillyHiatt,https://s3.amazonaws.com/bit-photos/large/7928...,Lilly Hiatt,https://s3.amazonaws.com/bit-photos/thumb/7928...,https://www.bandsintown.com/a/11801329?came_fr...,11801329,The new album 'Trinity Lane' is available Augu...,"Songs, Folk",...,https://www.bandsintown.com/e/1009523252?app_i...,Lilly Hiatt,Triple Door,United States,WA,Seattle,47.608433,-122.337944,Triple Door Seattle WA United States,38894
1048,1048,5b7718e6482deb63557f8d34,https://www.facebook.com/AMANDASHIRESMUSIC,https://s3.amazonaws.com/bit-photos/large/8692...,Amanda Shires,https://s3.amazonaws.com/bit-photos/thumb/8692...,https://www.bandsintown.com/a/184270?came_from...,184270,5'3 and hoping.,Americana,...,https://www.bandsintown.com/e/1009443259?app_i...,Amanda Shires,Triple Door,United States,WA,Seattle,47.608433,-122.337944,Triple Door Seattle WA United States,38894
1297,1297,5b771919482deb63557f8d67,NaN,https://s3.amazonaws.com/bit-photos/large/7881...,Rayland Baxter,https://s3.amazonaws.com/bit-photos/thumb/7881...,https://www.bandsintown.com/a/299253?came_from...,299253,It’s hard to pinpoint the moment that songs ar...,"Country, Alternative Country",...,https://www.bandsintown.com/e/1009690255?app_i...,Rayland Baxter,Tractor Tavern,United States,WA,Seattle,47.665771,-122.382645,Tractor Tavern Seattle WA United States,38718
2219,2219,5b771a17482deb63557f8e57,https://www.facebook.com/darinjonesmusic/,https://s3.amazonaws.com/bit-photos/large/8596...,Darin Jones & The Last Men Standing,https://s3.amazonaws.com/bit-photos/thumb/8596...,https://www.bandsintown.com/a/15183538?came_fr...,15183538,NaN,NaN,...,https://www.bandsintown.com/e/100221918?app_id...,Darin Jones & The Last Men Standing,Tractor Tavern,United States,WA,Seattle,47.668360,-122.376958,Tractor Tavern Seattle WA United States,38718
3551,3551,5b771ca9482deb63557f909e,https://www.facebook.com/heytherecolly/,https://s3.amazonaws.com/bit-photos/large/8365...,Colly,https://s3.amazonaws.com/bit-photos/thumb/8365...,https://www.bandsintown.com/a/4451373?came_fro...,4451373,Colly is an American rock band from Cincinnati...,"Cincinnati, OH",...,https://www.bandsintown.com/e/1009738556?app_i...,Colly,Central Saloon,United States,WA,Seattle,47.600594,-122.334379,Central Saloon Seattle WA United States,6744
3652,3652,5b771cc3482deb63557f90b1,ht